# Lab work on Static Context Header Compression (SCHC)

This is part of the material for the lecture given at Polytech' Grenoble on IoT Networks by Quentin Lampin, PhD.
This labwork is both a crash-course on the Static Context Header Compression (SCHC) protocol and a hands-on session. 

The following is the hands-on session which covers: 

- the analysis of a traffic capture between a CoAP client and server using IPv6.
- the creation of Compression/Decompression rules for this traffic.
- the evaluation of rules built by students using usual Compression metrics.
- the study of the underlying trade-offs of building Compression/Decompression rules, including a discussion on rule generalization, rules count and memory. 


## Pre-requisites

Students are assumed to have installed Wireshark, a working 3.11 Python interpreter and the packages listed in `requirements.txt`


## Pre-flight checklist - 5 mins

**directions**

Run the following Jupyter notebook cell, report any error that might occur.

In [ ]:
import pcapng
import microschc

## A look into the dataset - 15 mins

**directions**

Using Wireshark, open the dataset file: [`leshan-thermostat-readings.pcapng`](./dataset/leshan-thermostat-readings.pcapng) and answer the following questions.

- How many SCHC contexts?

- How many SCHC templates?

- What does each SCHC template correspond to?

- How many SCHC rules are necessary to compress all the packets of the dataset?


- For each template, list all constant fields

- What combination of Matching Operator and Compression/Decompression Action corresponds best to constant fields?

## Writing your own SCHC ruleset

microSCHC is a (micro)Python library that implements Compression and Decompression (C/D) functions of the SCHC protocol.

In the following part, you will write and evaluate SCHC C/D rules for a CoAP over UDP over IPv6 traffic, which is provided as a PCAPng capture file.

**directions**

Open the dataset and list packets contained within. (5mins)

**hints**

`tools.dataset` contains a helper function to open the dataset and list packets as `Buffer` items. 


microSCHC provides parsers (`PacketParser`) for IPv6, UDP and CoAP protocols that provide descriptions of packets (`PacketDescriptor`).

A `PacketDescriptor` includes a list of `FieldDescriptor` which provide an overview of the packets fields, most notably field IDs and field lengths.

**Directions**: 

In the following cell, 

- instantiate a packet parser for the stack IPv6/UDP/CoAP and parse the packets of the dataset. (5mins)
- parse the dataset (5mins)

**hints**:

- Relevant microSCHC packets are `microschc.protocol.registry` and `microschc.parser`.
- `tools.packet` contains a helper function for printing packet descriptors.


C/D rules are composed of Rule Field Descriptors. To be eligible for compression, every field of the parsed packet has to be matched to a field descriptor of the C/D rule.
It is therefore sensible to group packets with the same structure (template) before addressing them with C/D rules.

**directions**

- List packets templates of the dataset (5mins)
- print their structure (5mins)

**hints**

- `find_templates` in `tools.template` is your friend ;)
- `template_as_asciitable` in `tools.template` can be useful to print the templates.

Packets within the same template share the same structure, i.e. same fields in the same order, but one field may have different values from one packet to another.
As a first step, one can write a rule that compresses all constant fields for a given template. 


**directions**

- find all constant fields of the first template (#0) (10 mins)
- print them (42 sec)
- find all variable fields fo the first template (2mins)
- print them (42s)


**hints**

- `enumerate` is a nifty function that enumerate items of a list, providing the index of each element
- a `set` is a practical data structure to store unique values
- `filter` provides an elegant way to filter elements of an iterable 
- `fields_as_asciitable` in `tools.field` can be used to print fields

A SCHC C/D Rule is made of a list of field descriptors. a field descriptor has 4 main components: the Field ID (FID), the Matching Operator (MO), the Compression Decompression Action (CDA) and the Target Value (TV).

The FID corresponds to the Field Id of the field to compress, the MO indicates which packets can be compressed, the CDA tells how packets are compressed and the TV provides parameters to the CDA.

According to your knowledge, which MO/CDA couple should be used for constant fields?

Using microschc, create a rule matching the packet structure of template #0 that compresses every constant field.

** directions **

- import the fields definitions for IPv6, UDP and CoAP

** hints **

- protocol definitions are located in the `microschc.protocol` package
- rule field descriptor and rule descriptor are defined `microschc.rfc8724`

To produce a minimum SCHC Context, we need to add the no-compression rule, a rule that tells the residue is not compressed.

A no-compression rule is only defined by its nature and a Rule ID.

**directions**

create a SCHC context with your previous rule and the no-compression rule.

**hints**

- a no-compression rule is defined using:
```
no_compression_rule_descriptor: RuleDescriptor = RuleDescriptor(id=Buffer(content=b'\xXX', length=Y), nature=RuleNature.NO_COMPRESSION)
```

- the SCHC context is defined in the  `microschc.rfc8724extras` package



## Evaluating your SCHC rulesset

Now is time to evaluate your first ruleset. 

**directions**

- evaluate your ruleset against the dataset
- print Compression Factor (CF) statistics for your ruleset

**hints**

- the `tools.context` package contains a function to evaluate the context, in case you're running out of time.



## Going Further with SCHC

From now on, you can choose between the following instructions:

- augment your ruleset to address packets from the other templates
- improve your compression rule to address fields with few values (Match-Mapping)
- investigate MSB/LSB MO/CDA application
